In [1]:
TRAIN_DATA_PATH = "/home/haitien/Desktop/TwitterSentimentAnalysis_BigData20191/data/training.1600000.processed.noemoticon.csv"
TEST_DATA_PATH = "/home/haitien/Desktop/TwitterSentimentAnalysis_BigData20191/data/testdata.manual.2009.06.14.csv"

In [2]:
# Data column names
COL_NAMES = ["label", "id", "date", "query_string", "user", "text"]
# Text encoding type of the data
ENCODING = "iso-8859-1"

In [4]:
import pandas as pd  
import numpy as np
from matplotlib import pyplot as plt

df = pd.read_csv(TRAIN_DATA_PATH, header=None, names=COL_NAMES, encoding=ENCODING)

df.head()

,label,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.label.value_counts()

In [6]:
df.drop(['id','date','query_string','user'],axis=1,inplace=True)

In [7]:
df.head()

,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
!pip3 install bs4
!pip3 install nltk

In [23]:
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer
import re

pos_emoticons=["(^.^)","(^-^)","(^_^)","(^_~)","(^3^)","(^o^)","(~_^)","*)",":)",":*",":-*",":]",":^)",":}",
               ":>",":3",":b",":-b",":c)",":D",":-D",":O",":-O",":o)",":p",":-p",":P",":-P",":Þ",":-Þ",":X",
               ":-X",";)",";-)",";]",";D","^)","^.~","_)m"," ~.^","<=8","<3","<333","=)","=///=","=]","=^_^=",
               "=<_<=","=>.<="," =>.>="," =3","=D","=p","0-0","0w0","8D","8O","B)","C:","d'-'","d(>w<)b",":-)",
               "d^_^b","qB-)","X3","xD","XD","XP","ʘ‿ʘ","❤","💜","💚","💕","💙","💛","💓","💝","💖","💞",
               "💘","💗","😗","😘","😙","😚","😻","😀","😁","😃","☺","😄","😆","😇","😉","😊","😋","😍",
               "😎","😏","😛","😜","😝","😮","😸","😹","😺","😻","😼","👍"]

neg_emoticons=["--!--","(,_,)","(-.-)","(._.)","(;.;)9","(>.<)","(>_<)","(>_>)","(¬_¬)","(X_X)",":&",":(",":'(",
               ":-(",":-/",":-@[1]",":[",":\\",":{",":<",":-9",":c",":S",";(",";*(",";_;","^>_>^","^o)","_|_",
               "`_´","</3","<=3","=/","=\\",">:(",">:-(","💔","☹️","😌","😒","😓","😔","😕","😖","😞","😟",
               "😠","😡","😢","😣","😤","😥","😦","😧","😨","😩","😪","😫","😬","😭","😯","😰","😱","😲",
               "😳","😴","😷","😾","😿","🙀","💀","👎"]

# Emails
emailsRegex=re.compile(r'[\w\.-]+@[\w\.-]+')

# Mentions
userMentionsRegex=re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)')

#Urls
urlsRegex=re.compile(r'(f|ht)(tp)(s?)(://)(.*)[.|/][^ ]+')

#Numerics
numsRegex=re.compile(r"\b\d+\b")

punctuationNotEmoticonsRegex=re.compile(r'(?<=\w)[^\s\w](?![^\s\w])')

negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}

neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

emoticonsDict = {}
for i,each in enumerate(pos_emoticons):
    emoticonsDict[each]=' pos_emoticon '
    
for i,each in enumerate(neg_emoticons):
    emoticonsDict[each]=' neg_emoticon '
    
# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in emoticonsDict.items())
emoticonsPattern = re.compile("|".join(rep.keys()))

def processing_tweet(line):
    line = neg_pattern.sub(lambda x: negations_dic[x.group()], line)
    line = emoticonsPattern.sub(lambda m: rep[re.escape(m.group(0))], line)
    line = userMentionsRegex.sub(' ', line )
    line = emailsRegex.sub(' ', line )
    line = urlsRegex.sub(' ', line)
    line = numsRegex.sub(' ',line)
    line = punctuationNotEmoticonsRegex.sub(' ', line)
    line = re.sub(r'(.)\1{2,}', r'\1\1',line)
    line = line.lower()
    words_tokens = [token for token in TweetTokenizer().tokenize(line)]                    
    line = ' '.join(token for token in words_tokens)
    return line.strip()

print(processing_tweet("@haitien I'm love you xD"))

i m love you pos_emoticon


In [25]:
%%time
print ("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(0,1600000):
    if( (i+1)%10000 == 0 ):
        print ("Tweets %d of 1600000 has been processed" % (i+1))                                                                    
    clean_tweet_texts.append(processing_tweet(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 10000 of 1600000 has been processed
Tweets 20000 of 1600000 has been processed
Tweets 30000 of 1600000 has been processed
Tweets 40000 of 1600000 has been processed
Tweets 50000 of 1600000 has been processed
Tweets 60000 of 1600000 has been processed
Tweets 70000 of 1600000 has been processed
Tweets 80000 of 1600000 has been processed
Tweets 90000 of 1600000 has been processed
Tweets 100000 of 1600000 has been processed
Tweets 110000 of 1600000 has been processed
Tweets 120000 of 1600000 has been processed
Tweets 130000 of 1600000 has been processed
Tweets 140000 of 1600000 has been processed
Tweets 150000 of 1600000 has been processed
Tweets 160000 of 1600000 has been processed
Tweets 170000 of 1600000 has been processed
Tweets 180000 of 1600000 has been processed
Tweets 190000 of 1600000 has been processed
Tweets 200000 of 1600000 has been processed
Tweets 210000 of 1600000 has been processed
Tweets 220000 of 1600000 has been processed
Tweet

In [29]:
DATA_PATH = '/home/haitien/Desktop/TwitterSentimentAnalysis_BigData20191/data'
import os

clean_df = pd.DataFrame(clean_tweet_texts, columns=['text'])
clean_df['label'] = df['label']
clean_df.to_csv(os.path.join(DATA_PATH, 'cleaned_data.csv'),encoding='utf-8')

In [30]:
del clean_df
clean_df = pd.read_csv(os.path.join(DATA_PATH, 'cleaned_data.csv'), index_col=0)
clean_df.head()

/home/haitien/PycharmProjects/untitled4/venv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,text,label
0,- aww that s a bummer you shoulda got david ca...,0
1,is upset that he can not update his facebook b...,0
2,i dived many times for the ball managed to sav...,0
3,my whole body feels itchy and like its on fire,0
4,no it s not behaving at all i m mad why am i h...,0


In [31]:
clean_df[clean_df.isnull().any(axis=1)].head()

,text,label
208,NaN,0
249,NaN,0
398,NaN,0
430,NaN,0
1011,NaN,0


In [32]:
clean_df = clean_df.dropna()
clean_df.count()

text     1595942
label    1595942
dtype: int64

In [33]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

from pyspark import SparkContext
from pyspark.sql.context import SQLContext
try:
    sc = SparkContext('local[*]')
    sqlContext = SQLContext(sc)
except ValueError:
    warnings.warn("SparkContext already exists in this scope")

In [34]:
DATA = os.path.join(DATA_PATH, 'cleaned_data.csv')
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(DATA)

In [36]:
df.limit(10).toPandas()

,_c0,text,label
0,0,- aww that s a bummer you shoulda got david ca...,0
1,1,is upset that he can not update his facebook b...,0
2,2,i dived many times for the ball managed to sav...,0
3,3,my whole body feels itchy and like its on fire,0
4,4,no it s not behaving at all i m mad why am i h...,0
5,5,not the whole crew,0
6,6,need a hug,0
7,7,"hey long time no see yes .. rains a bit , only...",0
8,8,_k nope they did not have it,0
9,9,que me muera ?,0


In [38]:
from pyspark.sql.functions import rand, when, col
df = df.orderBy(rand()) \
                   .limit(100000) \
                   .withColumn("label", when(col("label") > 0, 1.0).otherwise(0.0)) \
                   .select(["label", "text"])

In [41]:
df.where(df.text.isNull()).show()

+-----+----+
|label|text|
+-----+----+
|  1.0|null|
|  0.0|null|
|  1.0|null|
|  1.0|null|
|  1.0|null|
|  1.0|null|
|  0.0|null|
|  0.0|null|
|  1.0|null|
|  0.0|null|
|  0.0|null|
|  1.0|null|
|  1.0|null|
|  1.0|null|
|  1.0|null|
|  1.0|null|
|  0.0|null|
|  1.0|null|
|  0.0|null|
|  1.0|null|
+-----+----+
only showing top 20 rows



In [42]:
df = df.na.drop(subset=["text"])

In [43]:
(train_set, val_set, test_set) = df.randomSplit([0.98, 0.01, 0.01], seed = 2000)

In [44]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])


pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
train_df.show(5)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|                text|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0| ! ! i am doing work|[!, !, i, am, doi...|(262144,[13396,24...|[0.81060438017967...|[0.69223827909894...|       0.0|
|  0.0|! ! i left my ted...|[!, !, i, left, m...|(262144,[24417,28...|[1.22422968232078...|[0.77280703827663...|       0.0|
|  0.0|! ! i never got y...|[!, !, i, never, ...|(262144,[24152,24...|[-1.0968725512473...|[0.25032634262170...|       1.0|
|  0.0|! ! i was puppysi...|[!, !, i, was, pu...|(262144,[18910,24...|[5.34855746627358...|[0.99526749776330...|       0.0|
|  0.0|   ! ! jailbreak ! !|[!, !, jailbreak,...|(262144,[28990,18...|[3.68314343040725...|[0.97547289234508...|       0.0|
+-----+-

In [45]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(val_df)

0.7943963114961768

In [46]:
test_df = pipelineFit.transform(test_set)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(test_df)

0.7806023458858791

In [47]:
pipelineFit.write().save("saved_model/model")

In [50]:
from pyspark.ml import PipelineModel
model = PipelineModel.read().load("saved_model/model")

In [51]:
model

PipelineModel_e648aa9f7f57

In [64]:
sqlContext = SQLContext(sc)
test = sqlContext.createDataFrame([
    (4, "i hate you"),
], ["id", "text"])

In [63]:
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, i hate you) --> prob=[0.7948757812412552,0.20512421875874484], prediction=0.000000
(5, i love you) --> prob=[0.13451663793386823,0.8654833620661319], prediction=1.000000


In [92]:
def predict(model, test_data):
    prediction = model.transform(test_data)
    selected = prediction.select("probability")
    selected = selected.toPandas()
    return selected['probability']

p = predict(model, test)

if p[0][0] <= 0.3:
    print("Pos")
if p[0][0] >= 0.7:
    print("Neg")
else:
    print("Neural")

    

Neg
